In [1]:
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
%matplotlib inline
pd.set_option("display.max_rows", 1000)

In [2]:
data = pd.read_csv("uiks-utf8.csv")

In [3]:
data.head()

,region_name,tik_name,uik_name,registered_voters,used_ballots,found_ballots,incorrect_ballots,yes_votes,no_votes,latitude,longitude
0,Республика Адыгея (Адыгея),1 Адыгейская,УИК №1,2277,1925,1925,0,1749,176,NaN,NaN
1,Республика Адыгея (Адыгея),1 Адыгейская,УИК №2,2694,2290,2290,7,2117,166,44.983019,39.103872
2,Республика Адыгея (Адыгея),1 Адыгейская,УИК №3,2865,2441,2441,11,2171,259,NaN,NaN
3,Республика Адыгея (Адыгея),1 Адыгейская,УИК №4,2054,1669,1669,8,1381,280,44.887695,39.202791
4,Республика Адыгея (Адыгея),1 Адыгейская,УИК №5,714,607,607,1,473,133,44.883646,39.183058


## Сумма голосов за/против

In [4]:
print(f"{data.yes_votes.sum()} Голосов За")
print(f"{data.no_votes.sum()} Голосов Против")

57071119 Голосов За
15347937 Голосов Против


## Ловушка удобных чисел

In [53]:
lucky_divider = 0.5

data['pct_turnout'] = (data.found_ballots / data.registered_voters) * 100
data['pct_yes']     = (data.yes_votes / data.found_ballots) * 100

data['lucky_turnout'] = data['pct_turnout'] % lucky_divider == 0
data['lucky_yes'] = data['pct_yes'] % lucky_divider == 0

data['lucky_all'] = data.apply(lambda x: x.lucky_turnout and x.lucky_yes, axis=1)

In [54]:
## Удалим из выборки потенциальные участки на предприятиях, военных частях и больницах, с небольшим числом зарегистрированных избирателей
## Как правило на таких участках 100% явка и это относительно нормально, и не свидетельствует о фальсификациях
data_ = data[data.registered_voters > 500]
print(f"Подвыборка из {(len(data_) / len(data)) * 100:.2f}% участков")

Подвыборка из 67.25% участков


### Явка

In [55]:
print("Самые часто-встречащиеся проценты явки")
print("-"*40)
turnout_counts = data_['pct_turnout'].value_counts()
for key, val in list(turnout_counts.iteritems())[:20]:
    print(f"{key:^20} | {val:^10}")

Самые часто-встречащиеся проценты явки
----------------------------------------
       100.0         |    212    
        75.0         |     57    
        90.0         |     39    
        50.0         |     33    
        80.0         |     31    
        95.0         |     25    
 85.71428571428571   |     25    
 66.66666666666666   |     20    
  90.9090909090909   |     19    
        40.0         |     17    
        87.5         |     16    
 83.33333333333334   |     14    
 71.42857142857143   |     14    
        70.0         |     14    
  92.3076923076923   |     14    
 77.77777777777779   |     12    
        60.0         |     12    
 91.66666666666666   |     12    
 80.95238095238095   |     11    
 86.20689655172413   |     11    


In [57]:
print(f"{len(data_[data_['pct_turnout'] % lucky_divider == 0])} участков, на которых процент явки нацело делится на {lucky_divider}")

612 участков, на которых процент явки нацело делится на 0.5


### Голоса ЗА поправки

In [58]:
print("Самые часто-встречащиеся проценты голосов ЗА поправки")
print("-"*40)
yes_counts = data_['pct_yes'].value_counts()
for key, val in list(turnout_counts.iteritems())[:20]:
    print(f"{key:^20} | {val:^10}")

Самые часто-встречащиеся проценты голосов ЗА поправки
----------------------------------------
       100.0         |    212    
        75.0         |     57    
        90.0         |     39    
        50.0         |     33    
        80.0         |     31    
        95.0         |     25    
 85.71428571428571   |     25    
 66.66666666666666   |     20    
  90.9090909090909   |     19    
        40.0         |     17    
        87.5         |     16    
 83.33333333333334   |     14    
 71.42857142857143   |     14    
        70.0         |     14    
  92.3076923076923   |     14    
 77.77777777777779   |     12    
        60.0         |     12    
 91.66666666666666   |     12    
 80.95238095238095   |     11    
 86.20689655172413   |     11    


In [59]:
print(f"{len(data_[data_['pct_yes'] % lucky_divider == 0])} участков, на которых процент голосов ЗА нацело делится на {lucky_divider}")

713 участков, на которых процент голосов ЗА нацело делится на 0.5


### Что чаще рисуют, голоса ЗА или явку(или и то и то)?

In [60]:
data_['lucky_turnout'].value_counts()

False    64463
True       612
Name: lucky_turnout, dtype: int64

In [61]:
data_['lucky_yes'].value_counts()

False    64362
True       713
Name: lucky_yes, dtype: int64

In [62]:
data_['lucky_all'].value_counts()

False    65066
True         9
Name: lucky_all, dtype: int64

Чаще рисуют явку, чуть реже голоса ЗА, но почти никогда не рисуют и то и другое одновременно

## Какие регионы попали в ловушку удобных чисел

In [63]:
data[data['lucky_turnout']]['region_name'].value_counts()

Нижегородская область                       224
Приморский край                             224
Территория за пределами РФ                  191
Сахалинская область                         140
Мурманская область                          139
Камчатский край                             103
Красноярский край                            98
Республика Башкортостан                      96
город Санкт-Петербург                        92
город Москва                                 92
Ставропольский край                          90
Республика Татарстан (Татарстан)             84
Республика Бурятия                           80
Республика Дагестан                          76
Архангельская область                        68
Краснодарский край                           67
Московская область                           58
Челябинская область                          57
Хабаровский край                             56
Омская область                               53
Иркутская область                       

In [64]:
data[data['lucky_yes']]['region_name'].value_counts()

Нижегородская область                       100
Приморский край                              85
Республика Дагестан                          83
Иркутская область                            76
Красноярский край                            75
Сахалинская область                          73
Республика Башкортостан                      69
Новосибирская область                        68
Московская область                           60
Свердловская область                         57
Республика Татарстан (Татарстан)             55
Оренбургская область                         53
Краснодарский край                           52
Омская область                               51
Алтайский край                               49
Челябинская область                          47
город Санкт-Петербург                        47
Архангельская область                        44
Ростовская область                           44
Саратовская область                          41
Волгоградская область                   

## Суммарно больше рисовали красивые числа в Приморском Крае(why?), Нижегородской области и Сахалинской области. Возможно рисовали и на зарубежных участках, но хз, они сами по себе странные.